In [45]:
import http.client, urllib.request, urllib.parse, urllib.error, base64, json
import traceback


headers = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': '', ## TODO
}

params = urllib.parse.urlencode({
    'model-version': 'latest',
    'showStats': 'true',
    'loggingOptOut': 'true',
    'opinionMining': 'true',
    'stringIndexType': 'TextElement_v8',
})

# https://{endpoint}/text/analytics/v3.1/sentiment[?model-version][&showStats][&loggingOptOut][&opinionMining][&stringIndexType]
def get_sentiment_analysis_cs(text: str):
    assert text

    body = {
        'documents': [
            {
                'id': "1",
                "language": "en",
                'text': text
            }
        ]
    }

    body_json = json.dumps(body)

    try:
        conn = http.client.HTTPSConnection('eastus.api.cognitive.microsoft.com')  ## TODO
        conn.request("POST", "/text/analytics/v3.1/sentiment?%s" % params, body_json, headers)
        response = conn.getresponse()
        data = response.read()
        conn.close()
        return(json.loads(data))
    except Exception:

        traceback.print_exc()

StatementMeta(new, 90, 28, Finished, Available)

In [19]:
project_name = ''

StatementMeta(new, 90, 2, Finished, Available)

In [33]:
ADLS_PATH = ""abfss://container@storage_account.dfs.core.windows.net/fb-scraper/" + project_name

posts_file = spark.read.option('header','true').option('delimiter',',').csv(ADLS_PATH + '/silver/posts_en.csv')
comments_file = spark.read.option('header','true').option('delimiter',',').csv(ADLS_PATH + '/silver/comments_en.csv')
comments_df = comments_file.select("Translation", 'CommentID', 'PostID').collect()

StatementMeta(new, 90, 16, Finished, Available)

In [50]:
des_df = comments_df
flatten = []
sentiment = []
positive = []
neutral = []
negative = []
c_ids = []
p_ids = []

for single_news in des_df:
    if single_news.asDict()["Translation"]:
        s = single_news.asDict()["Translation"]
        # print(s) # .replace('"', ' ').replace("'", '')
        r = get_sentiment_analysis_cs(s)
        flatten.append(s)
        c_id_ = single_news.asDict()["CommentID"]
        p_id_ = single_news.asDict()["PostID"]
        c_ids.append(c_id_)
        p_ids.append(p_id_)
        for i in r['documents']:
            sentiment.append(i['sentiment'])
            positive.append(i['confidenceScores']['positive'])
            neutral.append(i['confidenceScores']['neutral'])
            negative.append(i['confidenceScores']['negative'])

StatementMeta(new, 90, 33, Finished, Cancelled)

In [30]:
results = sqlContext.createDataFrame(zip(flatten, sentiment, positive, neutral, negative,  c_ids, p_ids), schema=['comments', 'sentiment', 'positive', 'neutral', 'negative', 'CommentID', 'PostID'])

results.show()

StatementMeta(new, 54, 30, Finished, Available)

+--------------------+---------+--------+-------+--------+
|            comments|sentiment|positive|neutral|negative|
+--------------------+---------+--------+-------+--------+
|I think I'm about...|  neutral|     0.0|    1.0|     0.0|
|Such a difference...| negative|    0.12|   0.22|    0.66|
|which in his case...|  neutral|    0.01|    0.9|    0.09|
|unfortunately you...| negative|     0.0|    0.0|     1.0|
|It was great, a t...|    mixed|    0.66|   0.05|    0.29|
|Wow! Super! Fruit...| positive|    0.95|   0.04|    0.01|
|        Thank you 😉| positive|    0.99|   0.01|     0.0|
|     Joanno brawo!👏|  neutral|    0.06|   0.92|    0.02|
|  Congratulation! 😄| positive|    0.73|   0.24|    0.03|
|Eventually, I ate...| negative|    0.37|   0.14|    0.49|
|Only 2 for now, b...|  neutral|    0.01|   0.73|    0.26|
|  Dominik Abramowicz|  neutral|    0.07|    0.9|    0.03|
|   Joanna Kucharczyk|  neutral|    0.06|   0.93|    0.01|
|Was it a take of ...| positive|    0.81|   0.17|    0.02|


In [32]:
results.write.format("csv").option("header","true").option("sep",",").mode("overwrite").save(ADLS_PATH + "/gold/" + 'comments_sentiment.csv')

StatementMeta(new, 54, 32, Finished, Available)